In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
# CDF Response Time Analysis Example 3
# Open network with two classes, comparing Fluid and JMT transient CDF

model = Network('myModel')

# Block 1: nodes
node = [None] * 4  # using 0-based indexing
node[0] = Source(model, 'Source')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)
node[3] = Sink(model, 'Sink')

# Block 2: classes
jobclass = [None] * 2  # using 0-based indexing
jobclass[0] = OpenClass(model, 'Class1', 0)
jobclass[1] = OpenClass(model, 'Class2', 0)

# Arrival processes
node[0].set_arrival(jobclass[0], Exp.fitMean(4.0))  # Source,Class1
node[0].set_arrival(jobclass[1], Exp.fitMean(4.0))  # Source,Class2

# Service processes
node[1].set_service(jobclass[0], Exp.fitMean(1.0))  # Queue1,Class1
node[1].set_service(jobclass[1], Exp.fitMean(1.0))  # Queue1,Class2

node[2].set_service(jobclass[0], Exp.fitMean(1.0))  # Queue2,Class1
node[2].set_service(jobclass[1], Exp.fitMean(1.0))  # Queue2,Class2

# Block 3: topology - complex routing matrix
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)  # Source->Queue1
P.set(jobclass[0], jobclass[1], node[1], node[2], 1.0)  # Queue1->Queue2 (class switch)
P.set(jobclass[1], jobclass[0], node[2], node[3], 1.0)  # Queue2->Sink

# Class 2 routing  
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # Source->Queue1
P.set(jobclass[1], jobclass[0], node[1], node[2], 1.0)  # Queue1->Queue2 (class switch)
P.set(jobclass[0], jobclass[1], node[2], node[3], 1.0)  # Queue2->Sink

model.link(P)

In [ ]:
# Solve with JMT solver - aligned with JAR test scenario  
jmtoptions = SolverJMT.defaultOptions()
jmtoptions.samples = int(10000)  # Aligned with JAR test: samples=10000
jmtoptions.seed = 23000  # Aligned with JAR test: seed=23000
RDsim = SolverJMT(model, jmtoptions).get_tran_cdf_resp_t()  # Using getTranCdfRespT like JAR test

# Solve with Fluid solver - aligned with JAR test scenario
options = SolverFluid.defaultOptions()  
options.iter_max = 300  # Aligned with JAR test: iter_max=300
RDfluid = SolverFluid(model, options).get_cdf_resp_t()

In [ ]:
# Plotting and comparison (note: Python equivalent of MATLAB plotting)
import matplotlib.pyplot as plt

num_queue_stations = model.get_number_of_stations() - 2  # Exclude Source and Sink
fig, axes = plt.subplots(num_queue_stations, 2, figsize=(12, 8))

# Ensure axes is always 2D array for consistent indexing
if num_queue_stations == 1:
    axes = axes.reshape(1, 2)
elif num_queue_stations == 2:
    # axes is already (2, 2), no reshape needed
    pass

for i in range(1, model.get_number_of_stations() - 1):  # stations 1 and 2 (queues)
    # Class 1 plot
    if RDsim[i][0] is not None and RDfluid[i][0] is not None:
        axes[i-1, 0].semilogx(RDsim[i][0][:, 1], 1 - RDsim[i][0][:, 0], 'r', label='sim')
        axes[i-1, 0].semilogx(RDfluid[i][0][:, 1], 1 - RDfluid[i][0][:, 0], '--', label='fluid')
        axes[i-1, 0].legend(loc='lower left')
        axes[i-1, 0].set_title(f'RespT Tail: Node {i}, Class 1')
    
    # Class 2 plot
    if RDsim[i][1] is not None and RDfluid[i][1] is not None:
        axes[i-1, 1].semilogx(RDsim[i][1][:, 1], 1 - RDsim[i][1][:, 0], 'r', label='sim')
        axes[i-1, 1].semilogx(RDfluid[i][1][:, 1], 1 - RDfluid[i][1][:, 0], '--', label='fluid')
        axes[i-1, 1].legend(loc='lower left')
        axes[i-1, 1].set_title(f'RespT Tail: Node {i}, Class 2')

plt.tight_layout()
plt.show()

# Calculate average response times from CDFs
AvgRespTfromCDFfluid = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
AvgRespTfromCDFsim = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(1, model.get_number_of_stations() - 1):  # only queue stations
    for c in range(model.get_number_of_classes()):
        # From Fluid CDF
        if RDfluid[i][c] is not None and len(RDfluid[i][c]) > 1:
            diffs = np.diff(RDfluid[i][c][:, 0])
            values = RDfluid[i][c][1:, 1]
            AvgRespTfromCDFfluid[i, c] = np.sum(diffs * values)
        
        # From Simulation CDF
        if RDsim[i][c] is not None and len(RDsim[i][c]) > 1:
            diffs = np.diff(RDsim[i][c][:, 0])
            values = RDsim[i][c][1:, 1]
            AvgRespTfromCDFsim[i, c] = np.sum(diffs * values)

print('AvgRespTfromCDFfluid =')
print(AvgRespTfromCDFfluid)
print('AvgRespTfromCDFsim =')
print(AvgRespTfromCDFsim)